In [1]:
import time

from dotenv import load_dotenv
from google import genai
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types
from pydantic import BaseModel, Field

In [2]:
load_dotenv()

retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [3]:
class FootballClubList(BaseModel):
    clubs: list[str] = Field(
        description="A list of exactly 10 major European football club names."
    )

In [4]:
client = genai.Client()

agent_sportscaster = Agent(
    name="sportscaster",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="You are a sportscaster.",
)
runner_sportscaster = InMemoryRunner(agent=agent_sportscaster)

agent_lister = Agent(
    name="lister",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="List 10 major European football clubs.",
    output_schema=FootballClubList,
)
runner_lister = InMemoryRunner(agent=agent_lister)


In [5]:
response_lister_raw = await runner_lister.run_debug(
    "List 10 major European football clubs"
)
response_lister = FootballClubList.model_validate_json(response_lister_raw[0].content.parts[0].text)
clubs =response_lister.clubs
print("clubs:", clubs)


 ### Created new session: debug_session_id

User > List 10 major European football clubs
lister > {
  "clubs": [
    "Real Madrid",
    "Barcelona",
    "Bayern Munich",
    "Manchester United",
    "Liverpool",
    "Juventus",
    "AC Milan",
    "Inter Milan",
    "Paris Saint-Germain",
    "Borussia Dortmund"
  ]
}
clubs: ['Real Madrid', 'Barcelona', 'Bayern Munich', 'Manchester United', 'Liverpool', 'Juventus', 'AC Milan', 'Inter Milan', 'Paris Saint-Germain', 'Borussia Dortmund']


In [6]:
print("Select a club:")
for i, club in enumerate(clubs, 1):
    print(f"{i}. {club}")

selection = int(input("Enter number: "))
human_club = clubs[selection - 1]
print(f"You selected: {human_club}")

Select a club:
1. Real Madrid
2. Barcelona
3. Bayern Munich
4. Manchester United
5. Liverpool
6. Juventus
7. AC Milan
8. Inter Milan
9. Paris Saint-Germain
10. Borussia Dortmund
You selected: Manchester United


In [7]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"What's the arch rival of {human_club}? Just give me the name of the club.",
)
computer_club = response.text
print(f"The arch rival of {human_club} is {computer_club}.")

The arch rival of Manchester United is Liverpool.


In [8]:
agent_computer = Agent(
    name="computer",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=f"You are the manager of the football club {computer_club}. You are responsible for the draft and the match between {human_club} and {computer_club}.",
)
runner_computer = InMemoryRunner(agent=agent_computer)

agent_administrator = Agent(
    name="administrator",
    model=Gemini(
        # model="gemini-2.5-flash-lite",
        model="gemini-2.5-pro",
        retry_options=retry_config,
    ),
    instruction=f"""
        You are the administrator of the draft and the match between {human_club} and {computer_club}.
        You have the capability to create pools of real players with varying skill levels.
        These players can be from any team in the world. They can be from the past or present.
        Once a player is shown, he must not be shown again.
        Player positions and stars:
            * Natural positions: Forward, Midfielder, Defender, Goalkeeper
            * Stars: 1–6
            * A play can have one or multiple natural positions
        When a player plays in a match, the following rules apply in terms of starts:
            * Natural position → full stars
            * Out-of-position → stars −1
            * Non-GK playing GK → stars = 1
            * GK playing other → stars = 1
    """,
)
runner_administrator = InMemoryRunner(agent=agent_administrator)

In [9]:
response_sportscaster_raw = await runner_sportscaster.run_debug(
    f"There is a draft and a match between {human_club} and {computer_club}. Briefly introduce the history of both clubs."
)


 ### Created new session: debug_session_id

User > There is a draft and a match between Manchester United and Liverpool. Briefly introduce the history of both clubs.
sportscaster > Hello everyone, and welcome to what promises to be an absolutely electrifying encounter! We're gearing up for a monumental clash between two titans of English football: **Manchester United** and **Liverpool**.

Now, before we dive into the action, let's take a moment to appreciate the sheer weight of history these two clubs carry.

**Manchester United**, hailing from the red half of Manchester, boasts a legacy built on soaring highs and heartbreaking lows. Founded in 1878 as Newton Heath LYR Football Club, they eventually became Manchester United in 1902. The Busby Babes era, tragically marred by the Munich Air Disaster in 1958, forged a spirit of resilience that saw them rise to become the first English club to win the European Cup in 1968. Under the legendary Sir Alex Ferguson, they entered an unprecedent

In [10]:
response_administrator = await runner_administrator.run_debug(
    """
    Create a pool of 16 real players with varying skill levels from 1 star to 6 stars.
    These players will be used for the draft.
    Although these players may play on a team in reality, for this draft they are all free agents and available to be selected.
    Each player's name, natural position(s), and star rating should be shown.
    The star ratings should be distributed as evenly as possible across the pool.
    The output must be a JSON object with the following format:

    {
        "players": [
            {"name": "player1", "positions": ["Forward"], "stars": 3},
            {"name": "player2", "positions": ["Midfielder"], "stars": 4},
        ]
    }
    """
)


 ### Created new session: debug_session_id

User > 
    Create a pool of 16 real players with varying skill levels from 1 star to 6 stars.
    These players will be used for the draft.
    Although these players may play on a team in reality, for this draft they are all free agents and available to be selected.
    Each player's name, natural position(s), and star rating should be shown.
    The star ratings should be distributed as evenly as possible across the pool.
    The output must be a JSON object with the following format:

    {
        "players": [
            {"name": "player1", "positions": ["Forward"], "stars": 3},
            {"name": "player2", "positions": ["Midfielder"], "stars": 4},
        ]
    }
    
administrator > ```json
{
    "players": [
        {
            "name": "Lionel Messi",
            "positions": ["Forward"],
            "stars": 6
        },
        {
            "name": "Zinedine Zidane",
            "positions": ["Midfielder"],
            "star